Goal / ideas

Successful ML organization

1. Make your requirements less dumb
2. Try to delete/remove the part/process
3. Simplify/optimize (don't optimize something that should not exist)
4. Accelerate cycle time
5. Automate

Looking for tech-aware management: if the product is good, we will eventually create value.

It is still early days for production-grade ML. Although the ecosystem of open-source and
commercial tools is changing fast, few patterns consistently emerged at mature ML-intensive
organizations. 

Data-driven companies come
in diverse shapes and cultures, yet they will often exhibit the below principles. They cluster
in three categories: People & Culture, Tools and Processes.

Tech-aware leadership
It is common and of capital importance to have science and technology advocates up in all
the leadership layers, to understand and support the scientific initiatives happening on the
field.

People and tools are not enough to deliver results consistently. Organizations need processes
to get in motion and create value. Processes drive productivity: they reduce the number of
ad-hoc meetings and thinking needed to address a situation. I detail below a non-exhaustive
list of baseline processes I believe to be factors of sucess in a data-driven organization.

DL applications : compromise between prediction speed and accuracy

https://github.com/visenger/awesome-mlops

## Feature store
the Feature Store refactors the monolithic end-to-end ML pipeline into a feature engineering and a model training pipeline.

## Word embeddings
Problème des *one-hot encoded* words:
* On se place en très haute dimension et on est *sparse*: les vecteurs sont de la taille du vocabulaire.
* Les représentations de deux mots différents sont par construction orthogonales: on a pas de notion de similarité entre les mots. 

Une idée qui s'avère puissante et fructueuse: représenter un mot par son contexte, i.e. un ensemble de mots alentours, le voisinage correspondant à une fenêtre de taille fixe par exemple. On va chercher à construire un *dense vector* de façon à ce qu'il soit par construction, similaire aux vecteurs représentant d'autres mots dans des contextes similaires.

Remarque: on parle parfois de *distributed representations*, le sens du mot étant réparti sur les $n$ (communément 300 à 1000) dimensions de l'*embedding*.

Word2vec 

Word2vec est un embedding particulier au texte. Son premier objectif est d'obtenir pour chaque mot d'un vocabulaire, une représentation sous la forme d'un vecteur (dense, contrairement à la représentation obtenue par one-hot encoding), ces représentations incorporant des relations entre mots, conservant des notions de similarité: les vecteurs de deux mots considérés comme "proches" auront des vecteurs considérés comme proches au sens d'une mesure de similarité (ex: cosine). On peut alors directement lire de ces représentations les similarités entre mots. Ces représentations contribuent notamment à faciliter l'apprentissage (elles sont de bien plus faibles dimension que le vocabulaire) et par les similarités qu'elles capturent, à améliorer les performances en généralisation: la performance ne sera que peu dégradée à la rencontre de mots jamais vus à l'entrainement mais pour lesquels le modèle a vu des mots similaires. 

Word2vec est un embedding donc un modèle. Le but du modèle est d'apprendre les représentations vectorielles de chacun des mots de façon supervisée. Word2vec est en fait un ensemble de deux modèles, aucun des deux correspondant à du deep learning (l'opération de softmax est parfois vue comme une "activation" ce qui peut conduire à voir les modèles du Word2vec comme des réseaux de neurones très simple et donc se rattachant au deep learning), les représentations apprises étant cependant très utilisées dans le domaine du NLP dont les modèles se rattachent de plus en plus au deep learning. 

Chacune des deux approches de Word2vec (Skip-gram et Continuous Bag of Words) produit pour chaque mot du vocabulaire deux représentations de même dimension: une en tant que center word (et dans le cas du skip gram qui est par construction efficace pour prédire les mots de son contexte) et une en tant que context word. 

Dans la pratique, un data scientist peut soit utiliser un Word2vec préentrainé sur un large corpus de texte ou choisir de le réentrainer sur un corpus plus proche de son cas d'usage afin de produire des représentations captant des relations entre mots spécifiques au domaine.

Dans ses deux approches, Word2vec est self-supervised: données et labels sont automatiquement dérivés du dataset (corpus de textes) sur lequel apprendre les embeddings. Chaque mot du corpus joue alternativement deux rôles, celui de center word et celui de context word. La génération du dataset d'entrainement à partir du corpus consiste à créer toutes les paires center word / context words à l'aide d'une fenêtre de longueur 2k+1, k étant un des principaux hyperparamètres du modèle avec la dimension des vecteurs appris. Pour chaque center word par définition situé au centre de la fenêtre on obtient 2k context words correspondant aux k mots précédant et suivant le center word.

Skip gram: 

Le modèle skip-gram cherche à estimer la probabilité conditionnelle P(context|center), c'est-à-dire la probabilité d'occurence du contexte étant donné un center word. On cherche pour chaque center word, à apprendre une représentation efficace pour prédire les mots apparaissant dans son contexte. Les représentations apprises à partir de cette problématisations sont intéressantes: elles sont de nature à capturer la similarité entre deux mots apparaissant dans des contextes similaires (?). Rejoint l'idée de si on arrive à prédire le contexte textuel dans lequel un mot apparaît, alors on en a compris, saisi le sens. 

Pour un vocabulaire de taille V et des représentations latentes prises de dimension d, on va apprendre deux matrices:
 - Une de taille (d, V) qui permet de passer d'une représentation one-hot d'un center word à sa représentation latente. Cette premère étape opère une réduction de dimension. Le type de réduction est par la forme de la transformation porteuse d'un "biais": il s'agit d'une projection. Sans doute que d'autres algorithmes/modélisations que word2vec sont capables d'opérer des réductions de dimension non linéaires et donc plus puissantes.
 - Une de taille (V, d) qui permet de passer de la représentation latente du center word à un vecteur de dimension V  dont chaque composante correspond à la similarité (produit scalaire) entre la représentation du center word et les représentation de chaque context word. Après softmax, ce vecteur nous donne une estimation de la loi binomiale P(w|center word). Soit le mot de contexte prédit correspond à celui associé à l'indice où se réalise le max, soit on peut échantillonner cette loi pour générer des mots de contexte.
 
Le modèle est simple: on postule que la probabilité P(w_i|cw) est une fonction de la mesure de similarité entre la représentation de w_i et celle de de cw qui sont de même dimension (leur produit scalaire). La fonction postulée étant un softmax qui permet d'obtenir des valeurs entre 0 et 1. On peut alors écrire la vraisemblance du modèle qu'on cherche simplement à maximiser sur les données (de façon équivalente la negative log likelihood correspond à la loss qu'on minimise). Le modèle est classiquement estimé par descente de gradient, opération pouvant être grandement accélérée par des procédures ad hoc (negative sampling, hierarchical softmax).

Continuous Bag Of Words 

Correspond à l'inverse de skip gram: on cherche à approcher P(center_word|context).

Pour prédire le center word, le modèle se sert des représentations latentes apprises pour chacun des context words. Le vecteur finalement utilisé pour la prédiction correspond à la moyenne arithmétique de ces context word vectors. Ce moyennage fait l'hypothèse implicite que l'ordre des mots du contexte n'influence pas la prédiction (bag of words assumption). De la même manière que pour le skip gram, la probabilité est supposée fonction (la fonction choisie étant un softmax) du produit scalaire (de la similiratité) entre ce bag-of-context-words vector et toutes les représentations apprises des center words du vocabulaire.

Par construction, CBOW cherche à prédire un unique (center) word à partir de plusieurs context words, là où skip gram fait l'inverse. Le premier converge/s'entraine donc beaucoup plus vite que le second.

Par construction:
- CBOW est meilleur à capturer les relations syntaxiques (relatives à l'arrangement grammatical des mots dans la phrase). Deux mots aux représentations similaires/proches seront donc souvent morphologiquement proches (ex: "cat" et "cats", la relation entre les deux étant ici bien grammaticale: le pluriel).
- Skip gram est meilleur à capturer les relations sémantiques (relatives à la signification). Deux mots aux représentations similaires/proches ne seront donc pas forcément morphologiquement proches (ex: "cat" et "dog" vus par le modèle comme similaires au sens où ce sont tous les deux des animaux).

Embedding: relation entre deux inputs à priori sans rapport directs et donc possiblement très éloignés dans leur espace d'origine => vont produire un output proche. Un embedding est in modèle qui va justement apprendre une représentation intermédiaire dans laquelle ces deux vecteurs sont proches de manière à ce que les prédictions qui en sont dérivées soient proches:
- CBOW : Quelle serait la relation entre deux contextes qui génèrent le même center word (?) Ou comment ça se traduirait sur les représentations intermédiaires apprises ?
- Skip gram : Quelle serait la relation entre deux mots (center words) qui genèrent le même contexte: plutôt une relation sémantique. 

Comportement vis à vis des mots rares/fréquents:
- CBOW: prompt à overfitter les mots fréquents: ils apparaissent plusieurs fois avec le même contexte. 
- Skip gram: moins sensible à l'overfitting sur les mots fréquents : même si présentés plusieurs fois pendant le training, ils le sont de façon individuelle. Même si ce modèle converge moins vite, à performance égale, il demande moins de documents/données (pour limiter l'impact de l'overfit des frequent words, on a besoin de remonter le nombre d'occurence de mots plus rares pour CBOW en lui passant un corpus plus large ?).

=> faire le raisonnement: qui joue quel rôle dans la loss, en quoi ça risque d'overfitter ? Points qui poussent à l'overfit ont un poids très important dans la loss soient parce que l'écarts d'un point individuel est toujours mportant (outlier) soit parce que qu'il y apparait beaucoup.

## *Language models*

Les *language models* sont une famille de modèles dont l'objectif est d'assigner une probabilité à un élément de textes ou plus formellement, à une séquence de *tokens*: $P(x_1, x_2, \dots, x_T)$. L'objectif de ces modèles est notamment pour une séquence donnée, de prédire le *token* qui suit. Les applications de ce principe sont très larges et compte par exemple la *sentence completion* dans un éditeur de texte, la *query completion* dans un moteur de recherche, etc.

En utilisant la définition d'une probabilité conditionnelle, la quantité $P(x_1, x_2, \dots, x_n)$ peut se réécrire:

$$P(x_1, x_2, \dots, x_T) = P(x_1).P(x_2|x_1).P(x_3|x2, x_1)\dots P(x_T|x_{T-1}, \dots, x_2, x_1)=\prod_{i=1}^{T}P(x_i|x_{i-1}, \dots, x_2, x_1)$$

Le but les *languages models* est d'estimer ces $P(x_i|x_{i-1}, \dots, x_2, x_1)$ pour tous les $x_i$ du vocabulaire, leur estimation permet à la fois d'utiliser ces modèles en prédiction (donner une probabilité à chaque *token* d'être le *token* suivant étant donné une séquence de *tokens*) comme d'affecter une probabilité $P(x_1, x_2, \dots, x_T)$ à une séquence comme le montre la formule ci-dessus.

Les *n-grams* models constitue une sous-famille de *language models* qui font l'hypothèse simplificatrice (*Markov assumption*) que le $t^e$ mot d'une séquence ne dépend que des $n-1$ mots précédents, i.e: $P(x_t|x_{t-1}, \dots, x_1) \approx P(x_t|x_{t-1}, \dots, x_{t-n-2})$. Dans le cas par exemple des *3-grams models*, chaque *token* d'une séquence ne dépend par hypothèse que des deux précédents, la formule de probabilité jointe ci-dessus se simplifie en: 

$$P(x_1, x_2, \dots, x_T) \approx \prod_{i=1}^{T}P(x_i|x_{i-1}, x_{i-2})$$

## Perplexity
La perplexité (*perplexity*) la métrique d'évaluation la plus répandue pour les *language models*. L'objectif de cette métrique est de comparer différents modèles entre eux.

Comme on va le voir, cette dernière est connectée à la cross-entropie qui est la *loss* la plus "naturellement" utilisée pour l'entrainement des *language models*. L'entrainement des *n-grams* models par exemple, consiste en effet à leur faire assigner une probabilité maximale au $n^e$ *token* d'une séquence connaissant les $n-1$ *tokens* de la séquence et ce pour l'ensemble des séquence à au plus $n$ éléments du corpus de texte d'entrainement (appelé aussi *teacher forcing*). Dans ce genre de problème de *soft classification* / d'entrainement de modèles probabilistes, la cross-entropie loss est traditionnellement utilisée.

Remarque: Le *teacher forcing* désigne le fait que d'entrainer le modèle d'une façon qui finalement le force à coller un peu trop aux données. Par exemple, si la séquence de travail est *The students opened their laptops* et qu'on cherche à prédire le *token* suivant sachant *The students opened*, on force le modèle à mettre toute la masse de sa pdf sur "their". Ainsi si le modèle output "the" ou "a" il sera pénalisé alors que ce n'est pas grammaticalement incorrect.

Remarque: La perplexité est une métrique d'évaluation intrinsèque. On distingue en effet dans l'évaluation de modèles, deux types d'évaluation:
* L'évaluation intrinsèque: Chaque modèle est évalué pour lui-même et non sur la base de sa performance à réaliser une tâche comme pour les métriques extrinsèques. 
* L'évaluation extrinsèque: Les différents modèles sont évalués sur la base de leur performance sur une tâche (ex: *machine translation*). La métrique d'évaluation correspond à celle utilisée pour évaluer la performance à effectuer cette tâche correctement. Cette méthode d'évaluation est "intéressée", on a l'idée d'évaluer les modèles dans leur capacité à résoudre la tâche qui nous intéresse. L'évaluation de métriques extrinsèques peut être particulièrement couteuse. On considère ces métriques comme plus informatives que les métriques intrinsèques.

Définition: 

Soit $\hat{y}_{w}^{(t)}$ la probabilité prédite à l'étape $t$ que le *token*/*mot* $w$ soit le $n+1^e$ *token* de la séquence. La cross-entropie *loss* du modèle à cette étape $t$ s'écrit: 

$$CE(y_{w}^{(t)}, \hat{y}_{w}^{(t)}) = -\sum_{w \in V}y_{w}^{(t)}log(\hat{y}_{w}^{(t)}) = -log(\hat{y}_{x_{t+1}}^{(t)})$$

Car $y_{w}^{(t)}$ est par définition nulle pour tous les mots différents de $x_{t+1}$ et égale à $1$ pour $x_{t+1}$.

La perplexité se définit comme la moyenne géométrique de l'inverse de la probabilité assignée au corpus de test par le *language model*:

$$perplexity = \frac{1}{P(x_1, x_2, \dots, x_N)}^{1/N} = \prod_{t=1}^{N}(\frac{1}{P(x_t | x_{t-1}, \dots, x_1)})^{1/N}$$

Remarques: 
* La moyenne géométrique effectue une normalisation par le nombre de mots du corpus. Cette normalisation est d'autant plus nécessaire que toutes choses égales par ailleurs, la perplexité sera par définition d'autant plus élevée que le corpus sera grand. La normalisation évite donc d'être favorisé par un corpus de test "petit".
* L'idée est que plus la perplexité est faible, meilleur est le modèle. En effet, plus la probabilité associé au corpus de test est élevée, moins cela signifie que le modèle est "surpris", "perplexe" quand on le lui présente compte tenu de ce qu'il a appris. Cette relation est traduite par le fait qu'on utilise une probabilité inverse dnas la définition de la perplexité.
* La perplexité ne peut jamais être rigoureusement nulle, même pour de très bons modèles on aura toujours une perplexité résiduelle. Cela peut facilement se comprendre: pour compléter la séquence "*He answered thank*", il est assez facile de le faire avec le mot "*you*", il n'y a presque jamais d'ambiguïté. En revanche, face à des phrases telles que "*He looked out of the window and saw a*", aucun *language models* ne peut vraiment bien savoir avec une bonne probabilité ce qui va suivre.

La perplexité permet de comparer la performance de différents *language models* sur la base d'un même corpus de test.

De la définition précédente, on voit immédiatement la connection avec la *cross-entropy loss*: la perplexité est simplement l'exponentielle de la *loss* sur le corpus de test. En effet, d'après ci-dessus: $\hat{y}_{x_{t+1}}^{(t)} = exp(-ln(b).CE(y_{w}^{(t)}, \hat{y}_{w}^{(t)})) = b^{-CE(y_{w}^{(t)}, \hat{y}_{w}^{(t)})}$ avec $b$ la base du logarithme utilisé et par définition: $\hat{y}_{x_{t+1}}^{(t)} = P(x_{t+1}| x_{t}, \dots, x_1)$ d'où:

$$perplexity = \prod_{t=0}^{N-1}(\frac{1}{\hat{y}_{x_{t+1}}^{(t)}})^{1/N} = \prod_{t=0}^{N-1} b^{CE(y_{x_{t+1}}^{(t)}, \hat{y}_{x_{t+1}}^{(t)})/N} = b^{\frac{1}{N}\sum_{t=0}^{N-1} CE(y_{x_{t+1}}^{(t)}, \hat{y}_{x_{t+1}}^{(t)})} = b^{loss}$$

Suivant le logarithme utilisé dans la définition de la cross entropie, on peut donc remarquer que la perplexité est égale dans sa définition à: $e^{loss}=e^{CE}$ si on utilise le logarithme naturel (base $e$) ou $2^{loss}=2^{CE}$ si on utilise le logarithme en base $2$ comme en théorie de l'information.

Remarque: La notion de perplexité n'a pas été introduite pour les *language models* (elle est même utilisée dans d'autres modèles comme le t-SNE). C'est en fait un concept qui provient de la théorie de l'information et les définitions sont plutôt à prendre dans l'aure sens: la perplexité se définit comme $2^{entropy}$, celle-ci étant utilisée comme *loss* dans les *language models* ce qui nous permet d'aboutir après développement à une définition de la perplexité pour ces modèles correspondant à la première formule donnée ci-dessus.

Pour approfondir les connections de la perplexité avec des concepts de théorie de l'information et notamment une interprétation de celle-ci comme *weighted branching factor*, voir https://towardsdatascience.com/perplexity-in-language-models-87a196019a94

L'interprétation comme *weighted branching factor* peut se comprendre simplement en partant de l'observation que la perplexité d'un dé à $k$ faces est égale à $k$ (comme dit plus haut, il y a une connection directe en entropie et perplexité et utiliser l'entropie d'une multinoulli équilibrée dans la definition de la perplexité donne ce résultat). Revenant aux *language models*, un modèle de perplexité 60 par exemple peut se voir comme aussi perplexe, aussi incertain au moment de prédire le prochain mot qu'on peut l'être face à l'anticipation du résultat d'un jet d'un dé à 60 faces. Dans l'absolu, cela peut sembler beaucoup, mais ramené à la taille du vocabulaire qui peut être très grande, cela nous donne une bonne appréciation de la performance du modèle et en particulier de sa capacité à être bien meilleur que le hasard (perplexité égale à la taille du vocabulaire).

## *Convolutional Neural Networks* (CNN)
Les CNN sont des réseaux ayant des propriétés permettant d'extraire et de préserver tout le long de la structure du réseau la structure spatiale des données qui lui sont présentés. A haut-niveau, l'architecture d'un CNN fait appel à trois types de couches spécialisées: 
* Les ***convolutional layers***,
* Les ***pooling layers***,
* Les ***fully-connected layers***.

**Ressources**:
* [Stanford CS231N | Lecture 5 | Convolutional Neural Networks (2017)](https://youtu.be/bNb2fEVKeEo)
* [Stanford CS231N | Lecture 9 | CNN Architectures (2017)](https://youtu.be/DAOcjicFr1Y)
* [Dive Into Deep Learning | CNN Chapter](http://d2l.ai/chapter_convolutional-neural-networks/index.html)

### *Convolutional layers*
Une *convolutional layer* se caractérise par: 
* Un *input* multi-dimensionnel, en général 3, on parle souvent d'***input volume***.
* Un ou plusieurs **filtres** appelé aussi *kernel* (généralement) **carrés, de même taille et ayant la même profondeur que l'*input***. Chacun de ces filtres va réaliser l'opération de convolution en "glissant" sur l'image. A chaque étape (*step*), on réalise le produit scalaire des paramètres du filtre (à apprendre) et de la zone de couverte de l'*input*, on y ajoute un biais et passe celà à une fonction d'activation pour obtenir l'activation correspondante.
* Une fonction d'activation.
* **Un *output volume* composé d'autant de *feature maps* que de filtres**. Chaque *feature map* ne comporte que deux dimensions (troisième dimension égale à 1). Une *feature map* (ou *activation map*) rassemble l'ensemble des activations du filtre auquel elle est associée. De ce point de vue, il y a autant de neurones dans une *convolutional layer* que de degrés de libertés dans une *feature map*. Ces *feature maps* servent d'*input* à la couche suivante.

Par exemple: un filtre de dimensions 5x5x3 appliqué (avec un *stride* de 1) à une image 32x32x3 va produire une *activation map* de dimension 28x28x1.

#### Qu'apprend une *convolutional layer* ?
Quand on s'intéresse à ce que recherchent et donc sur quoi se spécialisent les neurones d'une *convolutional layer* donnée, on voit que plus on est proche des données d'entrée (moins on est profond dans le réseau) plus les neurones se spécialisent dans la détection de *features* simples et élémentaires. Inversement, plus une couche est profonde, plus celle-ci se spécialise dans la détection d'éléments plus complexes, combinaisons d'éléments plus simples sur lesquels les couches antérieures se sont spécialisées, plus les *features* qu'elle tend à apprendre correspondent à des *higher level concepts*.

Comment sont produites les visualisations permettant de constater ces spécialisations ? Qu'y voit-on ? Pour chaque neurone d'une couche donnée, on va rechercher (par une technique ressemblant à la *back-propagation*) quel *input*, quelle image, maximise sont activation, ce qui est donc équivalent à rechercher l'image que ce neurone s'est spécialisé à détecter.

#### Hyperparamètres d'une *convolutional layer*
Les principaux hyperparamètres spécifiques d'une *convolutional layer*:
* **Le nombre de filtres carrés $K$**.
* **La taille des filtres $F$**. 
* La façon dont ils parcourent l'image avec **le *stride* $S$**, i.e. de combien de pixels on décale le filtre entre deux convolutions.
* Dans les cas où la taille de l'image, du filtre et le *stride* ne permettent pas de couvrir toute l'image sans en sortir, on peut réaliser **un *padding*** de l'image: on ajoute une bande sur le bord de $P$ pixels permettant de nouveau de couvrir l'intégralité de l'image pour les tailles d'input, de filtre et le *stride* qu'on s'est donné. Quels valeurs donner à des pixels supplémentaires ? En général $0$ (*zero-padding*) mais d'autres méthodes existent (ex: *mirroring*). 

Valeurs communes pour les différents hyperparamètres: 
* Nombre de filtres $K$: en général des puissances de $2$.
* Taille du filtre $F$: en général des nombres impairs: $1$, $3$, $5$, $7$. Cas particulier du filtre de taille $1$.
* *Stride* $S$: $1$, $2$, etc.
* *Padding* $P$: Peut être choisit adapté à la taille de l'*input*, du filtre et du *stride* de façon à ce que la taille des inputs de chaque couche ne diminue pas ou plus simplement de façon à faire en sorte que tout l*input* soit balayé. Sans *padding*, la taille des *feature maps* produites diminue régulièrement et souvent trop vite pour un bon apprentissage.

Remarque: **Les filtres sont tous de taille impaire**. C'est normal, chaque opération de convolution cherche à encoder une représentation du voisinage d'un pixel central (*source pixel* ou *anchor point*). Le carré du filtre doit donc être centré sur l'*anchor point* ce qui implique une taille de filtre impair. On peut se représenter les différentes taille de filtre comme égales à $2R+1$ avec $R$ le rayon définissant la taille du voisinage de l'*anchor point*. Chaque pixel de l'*input* peut avoir vocation à être l'*anchor point* d'une opération de convolution mais ce n'est pas toujours le cas: quand on introduit un *stride* $S>1$, chaque *anchor point* est séparé d'un ou plusieurs pixels.

Remarque: **Filtre de taille $F=1$**. Dans le cas particulier du filtre de taille $F=1$, on fait correspondre à l'information contenue dans tous les *channels* d'un pixel de l'*input* sans s'intéresser à son voisignage, un *pixel* de la *feature map* d'*output*. Cette technique permet de réduire le nombre de *channels* toutes choses égales par ailleurs. Par exemple, applique une *convolutional layer* composée de $16$ filtres $1x1$ à un input de taille $32x32x64$ va produire un volume de taille $32x32x16$. Les *convolutional layers* avec filtres de taille $1$ se rencontrent notamment dans les *deep architectures* où on les appelle *bottleneck layers*. Leur rôle est de réduire la profondeur et donc la dimension totale du volume afin de gagner en performance (diminuer le nombre d'opérations, de multiplications à réaliser dans les couches postérieures).

#### Déduire les dimensions de l'*output* volume de celles de l'*input volume* et des hyperparamètres
Pour un input carré de taille $N$, un nombre de filtres carrés $K$, de taille $F$, un *stride* $S$ et un padding $P$:
* **La profondeur du volume (i.e. le nombre de *feature maps*) correspond au nombre de filtres $K$**.
* **La dimension de chaque *feature map*: il s'agit du nombre de placements possible du filtre sur l'input**.
    * En négligeant le *padding* et en ne s'intéressant qu'à une dimension (par exemple la largeur, le filtre et l'input étant carré, le même raisonnement s'applique et donne le même résultat sur la hauteur), le nombre de placements possibles du filtre correspond au nombre de pas de taille $S$ qu'on peut faire sur une longueur de $N-F$ (puisque qu'on s'arrête quand notre filtre atteint le bout de l'input ou le dépasse) plus $1$ pour inclure la première position du filtre au compte. C'est à dire: $(N-F)/S + 1$. Dans le cas général, on prend la partie entière de $(N-F)/S$ pour tenir comte des cas où $N-F$ n'est pas un multiple de $S$ et où on laisse une partie de l'input non balayée.
    * Inclure le padding qu'on suppose le même dans les deux directions ne fait qu'agrandir l'image parcourue de $N$ à $N+2P$. La formule s'adapte simplement en:
    
    $$\frac{N-F+2P}{S}+1$$

Le choix de la valeur de *padding* $P$ est faite de façon à ce que $(N-F+2P)/S$ soit entier.

Pour plus d'exemples illustratifs sur "l'arithmétique convolutionnelle", voir notamment [ce papier](https://arxiv.org/pdf/1603.07285.pdf) et les animations de [ce repo](https://github.com/vdumoulin/conv_arithmetic).

#### Combien de neurones par *convolutional layer* ?
**Le nombre de neurones de la *convolutional layer* correspond à la taille de l'*output volume*.** Pour un filtre/une *feature map* donnée, on a autant de neurones que la taille de la *feature map* c'est-à-dire que de placements possibles du filtre sur l'input, c'est à dire que d'*anchor points*.

Une *convolutional layer* a la particularité que tous les neurones dédiés à la production d'une même *feature map* et donc accociés à un même filtre, partagent les mêmes paramètres (biais compris). Chaque neurone "surveille", se focalise sur une restriction de l'image d'entrée, sur le voisinage de l'*anchor point* qui lui est associé. 

Ainsi, le long de la profondeur dans l'*output volume*, tous les neurones n'intéressent en fait à la même zone de l'image, au même *anchor point* mais il n'y cherchent pas la même chose, ne se spécialisent pas de la même façon puisqu'ils sont associés à des filtres différents (ex: *horizontal edges* vs *vertical edges*).

Contrairement à l'approche naïve où on connecterait l'input à une *fully-connected layer* et dans laquelle chaque neurone de la couche se retrouve connecté à l'ensemble de l'image d'entrée, dans le cas d'une *convolutional layer*, chaque neurone n'est connecté qu'à une petite région de l'input (*locally connected*) qu'on appelle aussi *receptive field*. Par exemple, pour des filtres 3x3, le *receptive field* de chaque neurone de la couche est de taille 9 pixels. De plus dans le premier cas, on avait du mettre à plat (*flatten*, *stretch out*) l'image d'entrée dans un vecteur uni-dimensionnel, opération dans laquelle on perd la notion de voisinage spatial qui nous permet de capter la structure qu'on recherche dans les données.

Remarque: On peut montrer que si on *stack* trois *convolutional layers* chacune avec des filtres 3x3 et un *stride* de 1, l'*effective receptive field* des neurones de la dernière couche est de 7x7 (le neurone de la couche postérieur voit plus de l'input que chaque neurone de la couche antérieur car il "regarde" l'input via chacun des neurones de son propre *receptive field*). Ainsi un neurone d'une couche avec des filtres 7x7 a le même *effective receptive field* que le dernier neurone de trois couches avec des filtres 3x3. De là vient en partie la tendance à privilégier des filtres de petites tailles: on peut remplacer des couches à filtres de grande taille par plus de couches à filtres de petite taille. On a en plus l'avantage d'avoir moins de paramètres à estimer (dans cet exemple au moins) et l'accumulation des couches permet aussi de recourir à plus de non linéarité.

#### Combien de paramètres par *convolutional layer* ?
Soit $C$ le nombre de *channels* (la profondeur) de l'*input volume*. A chaque filtre correspondent donc $F.F.C$ paramètres plus un paramètre correspondant au biais. Pour une *convolutional layer* à $K$ filtres, le nombre de paramètres de la couche est donc:

$$K.(F.F.C + 1)$$

### *Pooling layers*
Ces couches **sans neurones** (parfois appelées *subsampling layers*) permettent de réduire la dimension des représentations produites par les *convolutional layers*. Elles opèrent dans les fait un *down-sampling* (sachant que jouer sur le *stride* dans une *convolutional layer* permet aussi de le faire) *feature map* par *feature map*, indépendemment des unes des autres. **Corolaire: la profondeur / le nombre de *channels* d'un volume est inchangé par passage à travers une *pooling layer*.**

Comme pour les *convolutional layers*, on va faire glisser une fenêtre, un filtre sur l'input sur laquelle va être réalisée l'opération de *pooling* (dans les faits, une opération d'agrégation, le plus souvent un max). On doit donc en particulier choisir une taille de filtre $F$, un *stride* $S$ et un *padding* $P$: 
* On ne veut en général pas d'*overlap* entre les fenêtres dans les opérations de *pooling*. **On prend chosit donc un *stride* tel que $S=F$**.
* Le (*zero*) *padding* n'est pas fréquemment utilisé pour ce type de *layer*.

Remarques:
* Les *pooling layers* **ne comportent pas de paramètres à estimer** (mais incluent évidemment des hyperparamètres): elles ne font qu'appliquer une même transformation à leur *input*.
* On préfère souvent le *max pooling* à l'*average pooling* notamment dans les cas de détection : on cherche à savoir si un ou des neurones se sont fortement activés dans une région particulière de l'image. Le max est bien plus de nature à propager cette information que la moyenne.
* Quand on craint d'introduire du bruit via le *padding*, on peut se rassurer par le fait que l'utilisation de *max pooling layers* à la suite de l'opération est de nature à en gommer une grande partie. Voir notamment la réponse à la deuxième question dans [cette réponse Stack Exchange](https://datascience.stackexchange.com/a/23186).
* De la même manière que les *convolutional layers* peuvent réduire plus ou moins fortement la taille de l'*output* comparée à celle de l'*input*, il faut faire attention dans le design d'une *pooling layer* à ne pas "trop" réduire la dimensions: on peut ne plus se donner assez de degrés de liberté pour conserver une représentation correcte de la structure des données.

### *Fully-connected layers*
Dans une *fully-connected layer*, chaque neurone de la couche est connecté à l'intégralité de l'*input volume* (ce qui a pose le problème du nombre de paramètres à estimer pour ces couches). En général utilisé comme dernière(s) couche(s) où ne se préoccupe plus de préserver la structure spatiale des données mais on cherche à agréger ces éléments pour produire un *output*, une variable de décision (ex: *softmax layer* pour la classification). A ce stade, à cette profondeur, les *inputs* des ces *fully-connected layers* correspondent au résultat cumulatif de ces traitements visant à extraire et conserver cette information spatiale.

### Architecture des CNN
A haut niveau, l'architecture de base d'un CNN est une succession de $M$ blocs chacun constitué de $N$ (autour de $5$) *convolutional layers* terminé par une *pooling layer*. Un tel réseau s'achève traditionnellement avec $K$ *fully-connected layers* (entre $0$ et $2$ la plupart du temps, en classification au moins une *softmax layer*). La tendence est toutefois de moins utiliser des *pooling* et des *fully-connected layers* (ces dernières faisant beaucoup augmenter le nombre de paramètres à estimer) au profit de *convolutional layers*.

#### Quelle empreinte mémoire ?
L'empreinte mémoire du réseau (en *forward-propagation*) est, pour une image, un seul *input* donné, égale à la somme des empreinte mémoire des *output volumes* de chaque couche plus l'*input* de la première couche, l'image d'entrée, à laquelle s'ajoute celle des coefficients (qui est notamment dimensionnante pour la taille de l'artefact issu du training).

Par exemple, pour un *output volume* de taille $(N,N,C)$ où chaque nombre est stocké sur $b$ bytes, l'empreinte mémoire est simplement égale à $N.N.C.b$ bytes.

Ainsi, même avec une image d'entrée de taille modeste (ex: 150KB), l'empreinte mémoire d'un réseau déjà un peu profond peut être de plusieurs dizaines de MB lors de la *forward propagation* seule.

Remarque: les *pooling layers* n'ont pas d'empreintes mémoire liés à leurs paramètres (elles n'en ont pas) mais elles ont bien une empreinte mémoire liée à l'*output* qu'elles produisent.

Sur les architectures classiques type VGG: on remarque que l'essentiel de la mémoire est consommée dans les premières *convolutional layers* où les *volumes* sont encore assez gros. A l'inverse, la majorité des paramètres à estimer se situe dans les *fully-connected layers* du fait de leur nature dense.

Globalement on recherche: la meilleure *accuracy*, la meilleure utilisation de la mémoire et la meilleure efficience computationnelle possible (nombre d'opérations). 